In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as parquet

In [2]:
# Where the downloaded data are
input_path = '/home/adeykin/projects/SNAHackaton'
# Where to store results
output_path = '/home/adeykin/projects/SNAHackaton/out'

In [26]:
test = parquet.read_table(input_path + '/imagesTest', \
                          columns = ['instanceId_userId', 'instanceId_objectId','ImageId']).to_pandas()
test.head(10)

,instanceId_userId,instanceId_objectId,ImageId
0,602,25708174,[d215ecd5da0a088db89b97db3b6e811a]
1,602,24393746,[79eacdd5e803ad80d1c62de4047d74ad]
2,803,25400331,[771591947cf0a72be0702fe53e83fe11]
3,803,36083673,[86a85b7f9a4f528e87dececd0b400fac]
4,803,33713302,[244ed11c75d01ab4c22ece5b6ab72f39]
5,3065,22946340,[269d9e263e34733e92944380a12b23ac]
6,3065,38325772,[056a349e7c71694baab197b79fe9b7d3]
7,3065,36426827,[24fec4db493603e1d546bcb6b71a5ae3]
8,3065,37549050,[9bc47732b71a66f12248649eef291ed5]
9,3065,20719945,[8e16231d4adb8e91d4fc13d949f1e2ce]


In [27]:
test['ImageId'] = test['ImageId'].map(lambda x: x[0])

In [28]:
#prediction = pd.read_pickle('predictLinear.pkl')
prediction = pd.read_pickle('predictGaussianBayes.pkl')

In [6]:
prediction.shape

(352505, 2)

In [29]:
df = pd.merge(left=test, right=prediction, left_on='ImageId', right_on='fileName')

In [67]:
test['ImageId'].unique().shape

(209109,)

In [68]:
prediction['fileName'].unique().shape

(352505,)

In [30]:
df = df[['instanceId_userId', 'instanceId_objectId', 'score']]

In [31]:
df['score'] = -df['score']

In [32]:
df.head()

,instanceId_userId,instanceId_objectId,score
0,602,25708174,-0.019393
1,14437766,25708174,-0.019393
2,5271680,25708174,-0.019393
3,5855396,25708174,-0.019393
4,7150973,25708174,-0.019393


In [33]:
df = df.sort_values(by=['instanceId_userId', 'score', 'instanceId_objectId'], ascending=True)

In [13]:
df.head()

,instanceId_userId,instanceId_objectId,score
126756,59,29096263,-0.019393
21422,59,39141251,-0.019393
228830,107,20368797,-0.019393
1175021,107,35377184,-0.019393
759771,107,35614366,-0.019393


In [21]:
df[df['instanceId_userId']==107]

,instanceId_userId,instanceId_objectId,score
296205,107,39090161,3.359316e-08
859207,107,10501436,1.762942e-06
434711,107,22664858,1.799480e-02
228830,107,20368797,1.939319e-02
1175021,107,35377184,1.939319e-02
759771,107,35614366,1.939319e-02
1154062,107,38038671,1.939319e-02


In [34]:
submit = df.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head(10)

instanceId_userId
59                                  [29096263, 39141251]
107    [20368797, 35377184, 35614366, 38038671, 22664...
158                                 [30223730, 33303760]
188                                 [30483180, 37981047]
254                                 [15200850, 24219820]
377    [16916552, 36170070, 38100230, 28882686, 37077...
527             [21421114, 38334144, 39110394, 39098017]
602                                 [24393746, 25708174]
803                       [36083673, 25400331, 33713302]
908                                 [21144274, 39092075]
Name: instanceId_objectId, dtype: object

In [35]:
submit.to_csv(output_path + "/gaussianBayesModelSubmit_original.csv.gz", header = False, compression='gzip')

In [36]:
#reverse
submit = df.groupby("instanceId_userId")['instanceId_objectId'].apply(list).apply(reversed).apply(list)
submit.head(10)

instanceId_userId
59                                  [39141251, 29096263]
107    [39090161, 10501436, 22664858, 38038671, 35614...
158                                 [33303760, 30223730]
188                                 [37981047, 30483180]
254                                 [24219820, 15200850]
377    [37077770, 28882686, 38100230, 36170070, 16916...
527             [39098017, 39110394, 38334144, 21421114]
602                                 [25708174, 24393746]
803                       [33713302, 25400331, 36083673]
908                                 [39092075, 21144274]
Name: instanceId_objectId, dtype: object

In [37]:
submit.to_csv(output_path + "/gaussianBayesModelSubmit_trueReverse.csv.gz", header = False, compression='gzip')